In [1]:
from model import Model

/Users/jackson.makl@dataiku.com/agent/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
m=Model()
m.chat("What Challenges Does Airbnb Face?",web=False,rag=True,tokens=100,use_gpt=False,use_sub_gpt=True,iters=3)
m.chat("What Challenges Does BMW Face?",web=False,rag=True,tokens=100,use_gpt=False,use_sub_gpt=True,iters=3)
m.chat("What can we learn from BMW & Airbnb here?",web=False,rag=True,tokens=100,use_gpt=False,use_sub_gpt=True,iters=3)



/Users/jackson.makl@dataiku.com/agent/rag.py:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
100%|██████████| 4/4 [00:00<00:00, 1169.23it/s]


Loading model meta-llama/Llama-3.2-3B-Instruct on mps


Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.19s/it]


Agent searching the web.....Agent searching the web.....
Researching: 
    
            Question:
            How does **Regulatory Issues**: The?

            Based On:
            What Challenges Does Airbnb Face?
            
    

Researching: 
    
            Question:
            What is Airbnb faces several?

            Based On:
            What Challenges Does Airbnb Face?
            
    
Agent searching the web.....
Researching: 
    
            Question:
            Why is **Pricing Transparency**: There's?

            Based On:
            What Challenges Does Airbnb Face?
            
    


100%|██████████| 4/4 [00:00<00:00, 900.07it/s]


Found 10 total results
After deduplication: 10 results
Fetching additional content from URLs...
Error fetching content from https://realtordr.com/the-rise-challenges-and-opportunities-of-airbnb-a-2023-review/: 403 Client Error: Forbidden for url: https://realtordr.com/the-rise-challenges-and-opportunities-of-airbnb-a-2023-review/
Found 10 total results
After deduplication: 10 results
Fetching additional content from URLs...
Found 5 total results
After deduplication: 5 results
Fetching additional content from URLs...
Error fetching content from https://www.sciencedirect.com/science/article/pii/S0007681318301915: 400 Client Error: Bad Request for url: https://www.sciencedirect.com/unsupported_browser
Researching: Airbnb regulatory challenges housing affordability legal restrictions city impact
Researching: Airbnb pricing transparency impact on customer trust business profitability hidden fees
Researching: Airbnb challenges regulatory compliance trust safety discrimination quality control

100%|██████████| 4/4 [00:00<00:00, 1077.53it/s]


Found 5 total results
After deduplication: 5 results
Fetching additional content from URLs...
Found 5 total results
After deduplication: 5 results
Fetching additional content from URLs...
Found 5 total results
After deduplication: 5 results
Fetching additional content from URLs...
USING CHAT GPT 4

Agent A1:  Great questions! For me, the intersection of personal safety and broader community impact is especially interesting. On one hand, stories about guests having bad experiences—things like property damage, scams, or even more severe incidents—really highlight how high the stakes can be when you mix strangers and private homes. At the same time, I’m fascinated by how Airbnb is reshaping entire neighborhoods, sometimes leading to higher rents or pricing out locals, which feels like a much bigger societal challenge.

Have you ever stayed at an Airbnb or considered renting your own place out? If so, did you feel that tension between the personal benefits and the potential risks, whether 

"The controversy surrounding Airbnb's pricing practices has shed light on several important lessons for both companies involved.\n\nFrom an operational perspective, **BMW**'s approach to managing its brand image through strategic partnerships highlights the importance of building relationships between brands and local communities. By partnering with influential figures like Jay-Z, BMW demonstrates its commitment to cultural relevance and community engagement. This strategy allows BMW to tap into emerging markets and stay relevant among younger demographics.\n\nIn contrast, **Airbnb**, which was criticized for allegedly perpet"

USING CHAT GPT 4
Researching: 
        Instructions:
          - Engage naturally as if you are having a real conversation.
          
          - If you need outside information to answer properly, say exactly: "I require information from the web"

          - Ask thoughtful questions to stay involved and deepen the discussion.

          - Feel free to explore your partner's ideas or shift the topic as you see fit.

          - You control the flow of the conversation — continue, pivot, or expand topics at your discretion.

          - Respond fully and clearly, using plain text only.          
        This policy requires BMW to do several key things to address the challenges it faces:

1. **Adapt Rapidly to New Technologies:** BMW must continually monitor and integrate the latest advancements in autonomous vehicle tech to avoid falling behind competitors.

2. **Stay Ahead of Rivals:** The company needs to innovate quickly and efficiently, since other companies are also racing to de

In [6]:
m.conscious

[{'role': 'user', 'content': 'What Challenges Does Airbnb Face?'},
 {'role': 'assistant',
  'content': 'Airbnb faces several challenges including:\n\n1. **Regulatory Issues**: The company has faced regulatory hurdles worldwide due to concerns over short-term rentals, tax evasion, and housing affordability.\n2. **Pricing Transparency**: There\'s ongoing debate about whether Airbnb should disclose its algorithms for setting host fees or if users can see more detailed breakdowns of costs associated with booking listings.\n\nRegarding price suggestions, when using "similar" as an option under search filters, Airbnb uses various data points such as location proximity'},
 {'role': 'user', 'content': 'What Challenges Does BMW Face?'},
 {'role': 'assistant',
  'content': "BMW faces various challenges due to its involvement with the Producer Responsibility policy implemented by Germany's government in 1990. This policy requires automobile manufacturers to collect used cars from consumers after 

In [5]:
m.subconscious

[[{'user': 'Great question! **Pricing transparency** is a key challenge for Airbnb because it directly impacts both the customer experience and Airbnb’s business strategy.\n\nHere’s why it’s such an issue:\n\n1. **Customer Trust:** Customers want to see the full cost of a stay before they book. In the past, many users were frustrated by hidden cleaning fees, service fees, and taxes that only appeared at checkout. Displaying the *total* price upfront builds trust and makes Airbnb seem more fair and honest.\n\n2. **Regulatory Pressure:** Governments are increasingly cracking down on hidden fees. Regulations now often require Airbnb to show all-inclusive prices, forcing the company to adapt.\n\n3. **Impact on Hosts**: When total costs are shown upfront, hosts often feel pressure to lower extra charges (like cleaning fees). This can reduce their earnings, which could make it harder for Airbnb to attract and keep hosts.\n\n4. **Business Challenge:** Displaying the true total price can make 

In [ ]:
m.subconscious


In [ ]:
m.chat("Explain money ball to me?",web=False,rag=True,tokens=150,use_gpt=False,use_sub_gpt=True,iters=2)

In [ ]:
m.subconscious

In [ ]:
m.subconscious